In [1]:
import pyteomics
import lxml
from pyteomics import mzml, mass
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from matplotlib import pyplot
import matplotlib
import seaborn as sns
import re
import math
import statsmodels
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from scipy.stats import f_oneway
from statsmodels.stats.multicomp import pairwise_tukeyhsd

In [2]:
df_pro=pd.read_csv(r'D:\\96_well\\csodiaq_results\\common_proteins.csv')
df_meta=pd.read_csv(r'D:\\96_well\\all_metabolites_unscaled.csv')

In [3]:
df_meta=df_meta.drop_duplicates(subset=['mz'],keep='first')

In [5]:
df_pro=df_pro.iloc[:,:83]
df_pro.set_index('Unnamed: 0',inplace=True)
new_dfpro=np.log2(df_pro)
new_dfpro

In [7]:
new_dfpro.columns=[item.split('/')[-1].split('.')[0] for item in new_dfpro.columns]

In [10]:
# apply standard scaler to the new dataframe 

ss = StandardScaler() 
df_ss = ss.fit_transform(new_dfpro)
dfpro_std=pd.DataFrame(df_ss,index=new_dfpro.index,columns=new_dfpro.columns)
dfpro_std

,BA1,BA2,BA3,BA4,BA5,BA6,BA8,BA9,BA10,BA11,...,BG3,BG4,BG5,BG6,BG7,BG8,BG9,BG10,BG11,BG12
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1/sp|Q13151|ROA0_HUMAN,-0.419862,-0.509531,-0.443065,-0.376354,-0.528730,-0.495093,-0.607515,-0.455514,-0.567042,-0.441701,...,-0.481138,-0.562861,-0.534626,-0.521348,-0.413663,-0.503991,-0.458139,-0.365285,-0.401619,-0.549408
1/sp|P25705|ATPA_HUMAN,1.292442,1.305703,1.200537,1.306900,1.315685,1.279125,1.335848,1.341425,1.215981,1.279351,...,1.332891,1.307595,1.343460,1.315538,1.346300,1.326967,1.291505,1.275103,1.259218,1.172761
1/sp|P17987|TCPA_HUMAN,1.157441,1.311948,1.157629,1.341931,1.501900,1.162090,1.176718,1.182790,1.163381,1.230998,...,1.493927,1.520830,1.353942,1.359419,1.613511,1.414099,1.457869,1.372922,1.466651,1.606053
1/sp|P06576|ATPB_HUMAN,1.309900,1.482894,1.377109,1.379143,1.338512,1.468378,1.358032,1.461026,1.373285,1.438550,...,1.253076,1.405747,1.367983,1.356687,1.268414,1.426175,1.413999,1.518927,1.497183,1.423773
1/sp|P62913|RL11_HUMAN,-0.136443,-0.641623,-0.149400,-0.688324,-0.035087,-0.558973,-0.356715,-0.157815,0.023582,-0.491995,...,-0.178390,-0.104637,-0.108939,-0.169631,-0.590971,-0.543082,-0.720816,-0.254539,-0.364011,0.039696
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1/sp|Q9NZL9|MAT2B_HUMAN,-1.472907,-1.547065,-1.429474,-1.342341,-1.396446,-1.355476,-1.439646,-1.369063,-1.362871,-1.379604,...,-1.157896,-1.252018,-1.498198,-1.127416,-1.096092,-1.376748,-1.147062,-1.247673,-1.067660,-1.324865
1/sp|P14174|MIF_HUMAN,2.503472,2.394960,2.476297,2.492811,0.220338,2.519988,2.289400,2.543517,2.391637,2.261505,...,2.111808,2.277465,2.151972,2.133002,2.142596,2.152637,2.300445,2.288111,2.218777,2.182901
1/sp|O00231|PSD11_HUMAN,-1.442387,-1.377624,-1.331076,-1.328406,-0.878526,-1.271352,-1.472322,-1.405344,-1.528806,-1.361596,...,-1.406705,-1.470400,-1.324628,-1.397706,-1.311183,-1.049420,-1.191263,-1.254661,-1.141077,-1.624574


In [12]:
# function for oneway_ANOVA analysis 
def oneway_ANOVA(result=dfpro_std):
    pval=[]
    for i in range(len(result)):
        aa=result.iloc[i,:11].tolist()
        bb=result.iloc[i,11:23].tolist()
        cc=result.iloc[i,23:35].tolist()
        dd=result.iloc[i,35:46].tolist() 
        ee=result.iloc[i,46:58].tolist()
        ff=result.iloc[i,58:70].tolist()
        gg=result.iloc[i,70:82].tolist()
        p_value=f_oneway(aa,bb,cc,dd,ee,ff,gg)[1]
        pval.append([i,p_value])
        fl = pd.DataFrame(pval)
        fl.columns = ['number', 'p_value']
    return(fl)

In [13]:
# make a dataframe for all p_values and corresponding protein name 
dfpro_pval=oneway_ANOVA(result=dfpro_std)
dfpro_pval['index_name']=dfpro_std.index.values.tolist()
dfpro_pval

,number,p_value,index_name
0,0,1.073709e-07,1/sp|Q13151|ROA0_HUMAN
1,1,8.468774e-03,1/sp|P25705|ATPA_HUMAN
2,2,1.092220e-05,1/sp|P17987|TCPA_HUMAN
3,3,4.412742e-03,1/sp|P06576|ATPB_HUMAN
4,4,5.765742e-01,1/sp|P62913|RL11_HUMAN
...,...,...,...
445,445,7.282941e-17,1/sp|Q9NZL9|MAT2B_HUMAN
446,446,6.628017e-01,1/sp|P14174|MIF_HUMAN
447,447,3.089690e-01,1/sp|O00231|PSD11_HUMAN
448,448,1.984754e-06,1/sp|Q00839|HNRPU_HUMAN


In [14]:
len([item for item in dfpro_pval['p_value'].tolist() if item <0.05])

306

In [15]:
FDR_res=statsmodels.stats.multitest.fdrcorrection(dfpro_pval['p_value'].tolist(), alpha=0.05, method='indep', is_sorted=False)[0]


In [16]:
# calculate the pvalues < 0.01 after FDR correction 

yuyu=[]
for item in range(len(FDR_res)):
    if FDR_res[item]== True:
        yuyu.append(item)
len(yuyu)

286

In [17]:
# locate the significant proteins from df_pval dataframe, aiming to find the protein names out 
sig_pros=dfpro_pval.loc[yuyu]
sig_pros


,number,p_value,index_name
0,0,1.073709e-07,1/sp|Q13151|ROA0_HUMAN
1,1,8.468774e-03,1/sp|P25705|ATPA_HUMAN
2,2,1.092220e-05,1/sp|P17987|TCPA_HUMAN
3,3,4.412742e-03,1/sp|P06576|ATPB_HUMAN
6,6,1.302602e-13,2/sp|Q14568|HS902_HUMAN/sp|P07900|HS90A_HUMAN
...,...,...,...
442,442,1.323286e-08,1/sp|P04406|G3P_HUMAN
444,444,5.908650e-03,3/sp|P61956|SUMO2_HUMAN/sp|Q6EEV6|SUMO4_HUMAN/...
445,445,7.282941e-17,1/sp|Q9NZL9|MAT2B_HUMAN
448,448,1.984754e-06,1/sp|Q00839|HNRPU_HUMAN


In [19]:
# save the raw values of significant proteins to df_pro_96well_raw 
df_pro_96well_raw =new_dfpro.loc[sig_pros['index_name'].tolist()]


#PCA analysis of significant proteins 



In [21]:
treatment=['Con']*11+['DMSO']*12+['DFO']*12+['TORI2']*11+['IDRIB']*12+['A939572']*12+['MG-132']*12
#PCA_pro_sig=dfpro_std.loc[sig_pros['index_name'].tolist()].T
PCA_pro_sig=dfpro_std.T

In [ ]:
# set the data soucrce
df =PCA_pro_sig
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)
# perform PCA with 2 principal components
pca = PCA(n_components=2)
pca.fit(X_scaled)
# transform the features to the new space
X_pca = pca.transform(X_scaled)
# create a new dataframe with the principal components and target variable
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
df_pca['target'] = treatment
# plot the results
import seaborn as sns
plt.figure(figsize=(8, 6), edgecolor='#04253a')
sns.scatterplot(x='PC1', y='PC2', hue='target', data=df_pca, s=180)
plt.yticks(fontsize=22)
plt.xticks(fontsize=22)
plt.xlabel('PC 1', fontsize=22)
plt.ylabel('PC 2', fontsize=22)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.2,fontsize=20)

In [ ]:
# standardize by rows again
scaler=StandardScaler()
dfpro_std2=dfpro_std
proname=dfpro_std2.T.columns
k=dfpro_std2.T
dfdf_pro=ss.fit_transform(k)
dfpro_stdbyrow=pd.DataFrame(dfdf_pro.T,index=dfpro_std2.index,columns=dfpro_std2.columns)
dfpro_stdbyrow

In [ ]:

#calculate the mean value of proteins from all repeats in each treatment
con=dfpro_stdbyrow.iloc[:,:11].mean(axis=1).tolist()
con_dmso=dfpro_stdbyrow.iloc[:,11:23].mean(axis=1).tolist()
DFO=dfpro_stdbyrow.iloc[:,23:35].mean(axis=1).tolist()
TORI2=dfpro_stdbyrow.iloc[:,35:46].mean(axis=1).tolist()
IDRIB=dfpro_stdbyrow.iloc[:,46:58].mean(axis=1).tolist()
N2356=dfpro_stdbyrow.iloc[:,58:70].mean(axis=1).tolist()
M7449=dfpro_stdbyrow.iloc[:,70:82].mean(axis=1).tolist()

newdict={'Con':con,'DMSO':con_dmso,'DFO':DFO,
        'TORI2':TORI2,'IDRIB':IDRIB,'A939572':N2356,
        'MG-132':M7449}

df_mean=pd.DataFrame.from_dict(newdict)
df_mean.index=dfpro_stdbyrow.index
df_mean  # dataframe of mean value of proteins 


In [26]:
def optimise_k_means(data,max_k):
    means=[]
    inertias=[]
    for k in range(1,max_k):
        kmeans=KMeans(n_clusters=k)
        kmeans.fit(data)
        means.append(k)
        inertias.append(kmeans.inertia_)
    fig=plt.subplots(figsize=(10,5))
    plt.plot(means,inertias)
    plt.xlabel('number of clusters')
    plt.ylabel('inertia')
    plt.show()

In [27]:
# select significant proteins from the mean value dataframe of proteins 

dfpro_sig=df_mean.loc[sig_pros['index_name'].tolist()]

In [ ]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(dfpro_sig[['Con', 'DMSO', 'DFO', 'TORI2', 'IDRIB', 'A939572', 'MG-132']])
dfpro_sig['kmeans']=kmeans.labels_
dfpro_sig

In [ ]:
df_aa=dfpro_sig.sort_values(by=['kmeans'])
df_bb=df_aa.T
df_heat=df_bb.iloc[:7,:]

plt.figure(figsize=(15,6),edgecolor="#04253a")
matplotlib.rcParams['font.family'] = "Arial"
sns.heatmap(df_heat,  vmin=-2, vmax=2,cmap='viridis',xticklabels=False,cbar_kws={"shrink": 0.7})
#cmap="RdBu_r"
plt.xticks([])
plt.yticks(rotation=30)
plt.yticks(fontsize=22)

In [31]:
sig_cluster3=dfpro_sig.loc[dfpro_sig['kmeans']==3]
sig_cluster2=dfpro_sig.loc[dfpro_sig['kmeans']==2]
sig_cluster1=dfpro_sig.loc[dfpro_sig['kmeans']==1]
sig_cluster0=dfpro_sig.loc[dfpro_sig['kmeans']==0]

In [ ]:
import matplotlib.pyplot as plt

x=['Con', 'DMSO', 'DFO', 'TORI2', 'IDRIB', 'A939572', 'MG-132']
# Create a dictionary of cluster names and corresponding data
cluster_data = {'proCluster 1': dfpro_sig.loc[list(sig_cluster0.index)],
                'proCluster 2': dfpro_sig.loc[list(sig_cluster1.index)],
                'proCluster 3': dfpro_sig.loc[list(sig_cluster2.index)],
                'proCluster 4': dfpro_sig.loc[list(sig_cluster3.index)]}

# Loop over the dictionary items to plot the data for each cluster
for title, data in cluster_data.items():
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.rcParams['font.family'] = 'Arial'
    for index in data.index:
            plt.plot(x,list(data.loc[index][:7]),color="#101E85",alpha=0.7,linewidth=2,label='proteins')     
    #plt.title(title, fontsize=24)
    #plt.xlabel('Treatment', fontsize=24)
    #plt.ylabel('Score', fontsize=24)
    plt.xticks(fontsize=20,rotation=30)
    plt.yticks(fontsize=22)
    plt.ylim(-2.2,2.2)
    plt.yticks([-2,-1,  0,  1,2])
    plt.tick_params(axis='x', which='both', length=10)
    plt.savefig(r'D:\\project7_SMAD\\{}.svg'.format(title),bbox_inches='tight')
    plt.show()

# clustering analysis of metabolites of 96well plate starts from here

In [34]:
newdf_meta=df_meta.set_index('mz')
newdf_meta

,A1,A10,A11,A12,A2,A3,A4,A5,A6,A8,...,G11,G12,G2,G3,G4,G5,G6,G7,G8,G9
mz,,,,,,,,,,,,,,,,,,,,,
110.07129,16.733969,15.349143,15.053247,16.226563,15.969387,15.826995,15.486207,14.849894,14.437622,14.916253,...,15.497166,16.191136,15.184953,15.332106,15.696153,16.354490,15.255987,15.792079,14.942148,14.973877
113.10736,12.931107,11.987264,11.713816,12.862250,11.591990,12.599448,11.751126,11.328675,12.392854,11.520619,...,12.504819,12.344296,11.754052,12.347068,13.596889,13.950374,11.781360,12.039262,12.279901,12.495355
114.12773,13.931107,12.942515,12.794213,12.933322,13.018200,13.456354,13.389362,13.028079,12.392317,12.569856,...,14.232571,13.800698,13.606174,13.970286,14.248224,14.058330,13.932953,14.469769,13.836642,14.498725
118.06511,14.720672,13.616549,13.947637,14.862541,14.170551,14.493730,14.134105,14.039605,13.313450,13.427837,...,14.585197,13.472945,13.852334,13.799686,13.804736,14.011926,13.810772,14.696968,13.863993,14.210671
135.04740,11.542548,10.884171,11.353698,11.753635,11.012625,11.255619,11.301496,10.542065,10.520619,11.215533,...,11.828533,11.194757,11.217352,11.216140,11.504819,11.265029,11.427313,11.266201,10.602699,10.559377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013.65083,11.431498,12.633449,12.203654,11.907642,12.380461,12.734074,12.505315,12.590353,12.238405,12.096386,...,12.683653,11.660442,12.952377,11.662668,12.505812,12.942881,12.556746,12.480538,13.338736,13.271901
979.50345,14.875557,14.088457,14.936178,15.134105,14.901244,14.726005,14.065079,14.717462,14.769011,14.370415,...,13.569856,13.191830,13.687157,13.659550,12.892543,12.919981,13.971184,13.563911,13.608486,12.183015
1009.51108,14.562123,13.126220,12.544964,13.625709,14.077650,14.653405,13.260332,13.954015,13.598053,12.984418,...,12.970645,11.964341,13.679480,12.419434,11.942148,12.642729,12.659773,12.690216,12.528698,11.940681


In [36]:
newdf_meta = newdf_meta.drop('D10', axis=1)

In [ ]:
# apply standard scaler to the new dataframe of metabolites 

ss = StandardScaler() 
df_sss = ss.fit_transform(newdf_meta)
df_std_meta=pd.DataFrame(df_sss,index=newdf_meta.index,columns=newdf_meta.columns)
df_std_meta

In [ ]:
# make a dataframe for all p_values and corresponding metabolites m/z  

df_pval_meta=oneway_ANOVA(result=df_std_meta)  # calculate p value of all metabolites 
df_pval_meta['index_name']=df_std_meta.index.values.tolist()
df_pval_meta

In [42]:
len([item for item in df_pval_meta['p_value'].tolist() if item <0.05])

5362

In [43]:
# FDR correction for all p_values of metabolites from Standard scaler less than 0.05 

FDR_res2=statsmodels.stats.multitest.fdrcorrection(df_pval_meta['p_value'].tolist(), alpha=0.01, method='indep', is_sorted=False)[0]


In [44]:
# calculate the pvalues < 0.05 after FDR correction 

yuyu2=[]
for item in range(len(FDR_res2)):
    if FDR_res2[item]== True:
        yuyu2.append(item)
len(yuyu2)

4307

In [ ]:
# locate the significant proteins from df_pval dataframe, aiming to find the protein names out 

sig_meta=df_pval_meta.loc[yuyu2]
sig_meta

In [46]:
# save the raw values of significant proteins to df_pro_96well_raw 
df_meta_96well_raw =newdf_meta.loc[sig_meta['index_name'].tolist()]
#df_meta_96well_raw.to_excel(r'D:\\project7_SMAD\\figure4_96well_multiomic_analysis\\df_meta_96well_raw.xlsx')


In [ ]:
# 按行进行标准归一化,不进行平均值计算
scaler=StandardScaler()
df_std_meta2=df_std_meta
metamz=df_std_meta2.T.columns
kk2=df_std_meta2.T
dfdf_meta=ss.fit_transform(kk2)


# 代谢物按行进行再标准化后的 dataframe
df_meta_stdbyrow=pd.DataFrame(dfdf_meta.T,index=df_std_meta2.index,columns=df_std_meta2.columns)
df_meta_stdbyrow


In [ ]:
['Con', 'DMSO', 'DFO', 'TORI2', 'IDRIB', 'A939572', 'MG-132']

In [ ]:

#calculate the mean value of metabolites from all repeats in each treatment
con_meta=df_meta_stdbyrow.iloc[:,:11].mean(axis=1).tolist()
dmso_meta=df_meta_stdbyrow.iloc[:,11:23].mean(axis=1).tolist()
DFO_meta=df_meta_stdbyrow.iloc[:,23:35].mean(axis=1).tolist()
TORI2_meta=df_meta_stdbyrow.iloc[:,35:46].mean(axis=1).tolist()
IDRIB_meta=df_meta_stdbyrow.iloc[:,46:58].mean(axis=1).tolist()
N2356_meta=df_meta_stdbyrow.iloc[:,58:70].mean(axis=1).tolist()
M7449_meta=df_meta_stdbyrow.iloc[:,70:82].mean(axis=1).tolist()


newdict_meta={'Con':con_meta,'DMSO':dmso_meta,'DFO':DFO_meta,
        'TORI2':TORI2_meta,'IDRIB':IDRIB_meta,'A939572':N2356_meta,
        'MG-132':M7449_meta}

df_mean_meta=pd.DataFrame.from_dict(newdict_meta)
df_mean_meta.index=df_meta_stdbyrow.index
df_mean_meta  # dataframe of mean value of proteins 

In [52]:
# select significant proteins from the mean value dataframe of proteins 

dfmeta_sig=df_mean_meta.loc[sig_meta['index_name'].tolist()]

In [ ]:
optimise_k_means(dfmeta_sig,10)  

In [ ]:

kmeans=KMeans(n_clusters=4)
kmeans.fit(dfmeta_sig[['Con', 'DMSO', 'DFO', 'TORI2', 'IDRIB', 'A939572', 'MG-132']])

dfmeta_sig['kmeans']=kmeans.labels_
dfmeta_sig

In [ ]:
df_aa_meta=dfmeta_sig.sort_values(by=['kmeans'])
df_bb_meta=df_aa_meta.T
df_heat_meta=df_bb_meta.iloc[:7,:]

plt.figure(figsize=(15,6),edgecolor="#04253a")
matplotlib.rcParams['font.family'] = "Arial"
sns.heatmap(df_heat_meta,  vmin=-2, vmax=2,cmap='viridis',xticklabels=False,cbar_kws={"shrink": 0.7})
#cmap="RdBu_r"
plt.xticks([])
plt.yticks(rotation=30)
plt.yticks(fontsize=22)

In [57]:
# metabolites sig of clusters 

#metasig_cluster4=dfmeta_sig.loc[dfmeta_sig['kmeans']==4]
metasig_cluster3=dfmeta_sig.loc[dfmeta_sig['kmeans']==3]
metasig_cluster2=dfmeta_sig.loc[dfmeta_sig['kmeans']==2]
metasig_cluster1=dfmeta_sig.loc[dfmeta_sig['kmeans']==1]
metasig_cluster0=dfmeta_sig.loc[dfmeta_sig['kmeans']==0]

In [ ]:
import matplotlib.pyplot as plt

x=['Con', 'DMSO', 'DFO', 'TORI2', 'IDRIB', 'A939572', 'MG-132']
# Create a dictionary of cluster names and corresponding data
cluster_data_meta = {'metaCluster 1': dfmeta_sig.loc[list(metasig_cluster0.index)],
                     'metaCluster 2': dfmeta_sig.loc[list(metasig_cluster1.index)],
                     'metaCluster 3': dfmeta_sig.loc[list(metasig_cluster2.index)],
                     'metaCluster 4': dfmeta_sig.loc[list(metasig_cluster3.index)],}
                     #'Cluster 5': dfmeta_sig.loc[list(metasig_cluster4.index)],}

# Loop over the dictionary items to plot the data for each cluster
for title, data in cluster_data_meta.items():
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.rcParams['font.family'] = 'Arial'
    for index in data.index:
            plt.plot(x,list(data.loc[index][:7]),color="#06d6a0",alpha=0.6,linewidth=2,label='proteins')     
    #plt.title(title, fontsize=24)
    #plt.xlabel('Treatment', fontsize=24)
    #plt.ylabel('Score', fontsize=24)
    plt.xticks(fontsize=20,rotation=30)
    plt.yticks(fontsize=22)
    plt.ylim(-2.5,2.5)
    plt.yticks([-2,-1,  0,  1,2])
    plt.tick_params(axis='x', which='both', length=10)

    plt.savefig(r'D:\\project7_SMAD\\{}.svg'.format(title),bbox_inches='tight')
    plt.show()

# PCA analysis of metabolites starts here  

In [ ]:
df_std_meta.T

In [59]:
treatment=['Con']*11+['DMSO']*12+['DFO']*12+['TORI2']*11+['IDRIB']*12+['A939572']*12+['MG-132']*12
PCA_meta_sig=df_std_meta.loc[sig_meta['index_name'].tolist()].T

In [ ]:
PCA_meta_sig

In [ ]:
# set the data soucrce
df =PCA_meta_sig

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df)

# perform PCA with 2 principal components
pca = PCA(n_components=2)
pca.fit(X_scaled)

# transform the features to the new space
X_pca = pca.transform(X_scaled)

# create a new dataframe with the principal components and target variable
df_pca = pd.DataFrame(X_pca, columns=['PC1', 'PC2'])
df_pca['target'] = treatment

# plot the results
import seaborn as sns
plt.figure(figsize=(8, 6), edgecolor='#04253a')
sns.scatterplot(x='PC1', y='PC2', hue='target', data=df_pca, s=180)
plt.yticks(fontsize=22)
plt.xticks(fontsize=22)
plt.xlabel('PC 1', fontsize=22)
plt.ylabel('PC 2', fontsize=22)
plt.legend(bbox_to_anchor=(1.01, 1), loc='upper left', borderaxespad=0.2,fontsize=20)

# combine two omic dataset and do multiomic analysis starts here

In [ ]:
result = pd.concat([dfmeta_sig, dfpro_sig], axis=0)
result=result.iloc[:,:7]
result

In [ ]:
kmeans=KMeans(n_clusters=4)
kmeans.fit(result[['Con', 'DMSO', 'DFO', 'TORI2', 'IDRIB', 'A939572', 'MG-132']])

result['kmeans']=kmeans.labels_
result

In [61]:
newresult=pd.read_excel(r'D:\\project7_SMAD\\df_heat_96well.xlsx',index_col=0)

newresult

In [63]:
result2=newresult.iloc[:,:7]

In [ ]:
dualsig_cluster3=newresult.loc[newresult['kmeans']==3]
dualsig_cluster2=newresult.loc[newresult['kmeans']==2]
dualsig_cluster1=newresult.loc[newresult['kmeans']==1]
dualsig_cluster0=newresult.loc[newresult['kmeans']==0]

# Create a dictionary of cluster names and corresponding data
cluster_data = {'DualCluster 1': result2.loc[list(dualsig_cluster0.index)],
                'DualCluster 2': result2.loc[list(dualsig_cluster1.index)],
                'DualCluster 3': result2.loc[list(dualsig_cluster2.index)],
                'DualCluster 4': result2.loc[list(dualsig_cluster3.index)]}
x=['Con', 'Dmso', 'Dfo', 'Tori2', 'Isrib', 'A939572','MG-132']
# Loop over the dictionary items to plot the data for each cluster
for title, data in cluster_data.items():
    plt.figure(figsize=(6, 5), edgecolor='#04253a')
    plt.rcParams['font.family'] = 'Arial'
    num1,num2=[],[]
    for index in data.index:
        if isinstance(index, float):
            plt.plot(x,list(data.loc[index]),color="#00B050",alpha=0.6,linewidth=2,label='metabolites')
            num1.append(index)
        if isinstance(index, str):
            plt.plot(x,list(data.loc[index]),color="#7030A0",alpha=0.6,linewidth=2.2,label='proteins') 
            num2.append(index)
    print([len(num1),len(num2)])
    #plt.title(title, fontsize=24)
    #plt.xlabel('Treatment', fontsize=24)
    #plt.ylabel('Score', fontsize=24)
    plt.xticks(fontsize=24,rotation=30)
    plt.yticks(fontsize=24)
    plt.ylim(-2.5,2.5)
    plt.yticks([-2,-1,  0,  1,2])
    plt.tick_params(axis='x', which='both', length=9)
    plt.savefig(r'D:\\project7_SMAD\\figure4_96well_multiomic_analysis\\{}.svg'.format(title),bbox_inches='tight')
    plt.show()

# plot typical metabolites and protein here

In [83]:
treatment=['Con']*11+['DMSO']*12+['DFO']*12+['TORI2']*11+['IDRIB']*12+['A939572']*12+['MG-132']*12

def getdf(proname='1/sp|P04406|G3P_HUMAN',df=dfpro_stdbyrow):    
    A=df.iloc[:,:11].loc[proname].to_frame()
    A.columns=['Value']  
    B=df.iloc[:,11:23].loc[proname].to_frame()
    B.columns=['Value']   
    C=df.iloc[:,23:35].loc[proname].to_frame()
    C.columns=['Value']
    D=df.iloc[:,35:46].loc[proname].to_frame()
    D.columns=['Value']
    E=df.iloc[:,46:58].loc[proname].to_frame()
    E.columns=['Value']
    F=df.iloc[:,58:70].loc[proname].to_frame()
    F.columns=['Value']    
    G=df.iloc[:,70:82].loc[proname].to_frame()
    G.columns=['Value']      
    made=pd.concat([A,B,C,D,E,F,G])
    made['treatment'] = treatment
    return(made)
def plot_fig(listname=['proname']):
    for number in listname:
        plt.figure(figsize=(11,3))
        dataset=getdf(proname=number,df=dfpro_stdbyrow)
        #palette = ['#9B3A4D', '#E2AE79', '#497EB2','#D0DCAA', '#8CBDA7', '#68AC57','#70A0AC']
        ax = sns.boxplot(x="treatment", y="Value",width=0.5, data=dataset,palette='viridis',showfliers=False)
        ax = sns.stripplot(x="treatment", y="Value", data=dataset, color="grey",size=8)
        ax.set_xlabel("")
        ax.set_ylabel("Score", fontsize=24)
        ax.tick_params(axis='both', labelsize=20)
        ax.set_xticklabels(ax.get_xticklabels(),rotation=0)
        plt.title(number.split("|")[1])
        plt.show()
def plot_fig2(listname=name,dftar=dfpro_stdbyrow):
    plt.figure(figsize=(11,3))
    dataset=getdf(proname=listname,df=dftar)
    #palette = ['#9B3A4D', '#E2AE79', '#497EB2','#D0DCAA', '#8CBDA7', '#68AC57','#70A0AC']
    ax = sns.boxplot(x="treatment", y="Value",width=0.5, data=dataset,palette='viridis',showfliers=False)
    ax = sns.stripplot(x="treatment", y="Value", data=dataset, color="grey",size=8)
    ax.set_xlabel("")
    ax.set_ylabel("Score", fontsize=24)
    ax.tick_params(axis='both', labelsize=20)
    ax.set_xticklabels(ax.get_xticklabels(),rotation=0)
    plt.title(name.split("|")[1])
    plt.show()

In [75]:
pros=['1/sp|P04406|G3P_HUMAN',
      '1/sp|Q9NZL9|MAT2B_HUMAN',
      '1/sp|P00338|LDHA_HUMAN',
     '3/sp|Q6ZMR3|LDH6A_HUMAN/sp|Q9BYZ2|LDH6B_HUMAN/sp|P00338|LDHA_HUMAN']

In [110]:
[item for item in dfpro_stdbyrow.index if 'LDH' in item]
    

['1/sp|P00338|LDHA_HUMAN',
 '1/sp|P09622|DLDH_HUMAN',
 '3/sp|Q6ZMR3|LDH6A_HUMAN/sp|Q9BYZ2|LDH6B_HUMAN/sp|P00338|LDHA_HUMAN',
 '1/sp|P07195|LDHB_HUMAN']

In [ ]:
for name in pros:
    plot_fig2(listname=name,dftar=dfpro_stdbyrow)
    k=name.split("|")[1]
    plt.savefig(r'D:\\project7_SMAD\\figure4_96well_multiomic_analysis\\sig_pro\\{}.svg'.format(k),bbox_inches='tight')

In [ ]:
list2=['2/sp|Q8TAA3|PSMA8_HUMAN/sp|O14818|PSA7_HUMAN', '1/sp|P51665|PSMD7_HUMAN']
plot_fig(listname=list2)